In [24]:
%%sh
## Installing dependencies from git.
pip install folium
rm -fr data/
mkdir -p data
cd data
wget https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/california-counties.geojson

--2018-02-06 09:13:32--  https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/california-counties.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257380 (251K) [text/plain]
Saving to: ‘california-counties.geojson’

     0K .......... .......... .......... .......... .......... 19% 3.27M 0s
    50K .......... .......... .......... .......... .......... 39% 8.99M 0s
   100K .......... .......... .......... .......... .......... 59% 19.7M 0s
   150K .......... .......... .......... .......... .......... 79% 9.94M 0s
   200K .......... .......... .......... .......... .......... 99% 15.9M 0s
   250K .                                                     100% 2570G=0.03s

2018-02-06 09:13:32 (7.96 MB/s) - ‘california-counties.geojson’ saved [257380/257380]



In [11]:
import os,boto3,time
import folium
from io import StringIO
import pandas as pd

print(folium.__version__)


0.5.0


In [25]:
results_bucket='aws-athena-query-results-<accountid>-us-east-1'
region='us-east-1'
def executeAthenaQuery(database,queryString):
   client = boto3.client(service_name='athena', region_name=region)
   s3c = boto3.client('s3')

   response = client.start_query_execution(
      QueryString=queryString,
      QueryExecutionContext={
        'Database': database
      },
      ResultConfiguration={
        'OutputLocation': 's3://'+results_bucket
      }
   )
   queryStatus='RUNNING'
   while queryStatus=='RUNNING':
      time.sleep(10)  
      response1=client.get_query_execution(QueryExecutionId=response['QueryExecutionId'])
      #print response
      queryStatus=response1['QueryExecution']['Status']['State']
      print (queryStatus)  
   if queryStatus=='SUCCEEDED':
      query_id=response['QueryExecutionId']
      results_key=query_id+'.csv'
      obj = s3c.get_object(Bucket=results_bucket, Key=results_key)
      content=obj['Body'].read()
      csv=StringIO(content.decode("utf-8"))
      df = pd.read_csv(csv, sep=",")
      return df
   return None

In [26]:
counties_geo = os.path.join('data', 'california-counties.geojson')

k = folium.Map(
    location=[37.5, -120],
    zoom_start=6,
    tiles='OpenStreetMap'
)

folium.GeoJson(
    counties_geo,
    name='geojson').add_to(k)

k

In [27]:
database='geospatial'
query='SELECT counties.name as county,\
          COUNT(*) as cnt \
          FROM counties \
          CROSS JOIN earthquakes \
          WHERE ST_CONTAINS (counties.boundaryshape, ST_POINT(earthquakes.longitude, earthquakes.latitude)) \
          GROUP BY  counties.name \
          ORDER BY  cnt DESC'
df=executeAthenaQuery(database,query)
df

RUNNING
RUNNING
RUNNING
SUCCEEDED


county  cnt
0              Kern   36
1    San Bernardino   35
2          Imperial   28
3              Inyo   20
4       Los Angeles   18
5         Riverside   14
6          Monterey   14
7       Santa Clara   12
8        San Benito   11
9            Fresno   11
10        San Diego    7
11       Santa Cruz    5
12          Ventura    3
13  San Luis Obispo    3
14           Orange    2
15        San Mateo    1

In [28]:
k = folium.Map(
    location=[37.5, -120],
    zoom_start=6,
    tiles='OpenStreetMap'
)

k.choropleth(
    counties_geo,
    data=df,
    columns=['county', 'cnt'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Earthquakes in CA'
)

k